# Convolutional Neural Network Using Mel Spectrogram Classifying Gender

In [1]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
audio_data = np.load("../data/audio_data.npy")
labels = np.load("../data/wav_labels.npy")

# labels: modality-vocal channel-emotion-emotional intensity-statement-repetition-actor
# emotions: 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# odd number actors = male, even = female

# 1440 files: 24 speakers, 60 recordings per speaker
audio_data = audio_data.reshape(1440, 9480)

features = []

for i in range(1440):
    
    if (labels[i][6]%2 == 0):
        label = "Female"
    else:
        label = "Male"

    if (labels[i][2] == 1):
        em = 0
    elif (labels[i][2] == 2):
        em = 1
    elif (labels[i][2] == 3):
        em = 2
    elif (labels[i][2] == 4):
        em = 3
    elif (labels[i][2] == 5):
        em = 4
    elif (labels[i][2] == 6):
        em = 5
    elif (labels[i][2] == 7):
        em = 6
    elif (labels[i][2] == 8):
        em = 7
    
    features.append([audio_data[i], label, em])
    
feature_df = pd.DataFrame(features, columns = ["mfcc", "gender", "emotion"])

feature_df.head()


,mfcc,gender,emotion
0,"[-710.0553588867188, -709.9026489257812, -711....",Female,0
1,"[-547.765625, -548.0353393554688, -548.6129760...",Female,2
2,"[-616.4595336914062, -615.7564697265625, -615....",Female,7
3,"[-739.8626098632812, -738.7739868164062, -735....",Female,1
4,"[-698.0630493164062, -697.3838500976562, -696....",Female,4


In [3]:
#split data - males vs females
mal = feature_df.loc[feature_df['gender'] == "Male"]
fem = feature_df.loc[feature_df['gender'] == "Female"]

## ===== Females ===== ##
X_females_unscaled = np.array(fem.mfcc.tolist())
y_females = np.array(fem.emotion.tolist())

scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X_females_unscaled)
X_females_scaled = scaler.transform(X_females_unscaled)

#20-80 train-test split
X_train_females, X_test_females, y_train_females, y_test_females = train_test_split(X_females_scaled, y_females, test_size=0.20, random_state=0)

In [4]:
# Reshape for CNN input
X_train_females = np.array([x.reshape( (20, 474, 1) ) for x in X_train_females])
X_test_females = np.array([x.reshape( (20, 474, 1) ) for x in X_test_females])

# One-Hot encoding for classes
y_train_females = np.array(to_categorical(y_train_females, 8))
y_test_females = np.array(to_categorical(y_test_females, 8))

In [5]:
model = Sequential()

model.add(Conv2D(64, (5,5), input_shape=X_test_females.shape[1:], kernel_regularizer=l2(l=0.01)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(24, (4,4)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(rate=0.2))

model.add(Conv2D(24, (3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(rate=0.2))

model.add(Conv2D(80, (2,2)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(rate=0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('softmax'))

In [6]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy']
)

model.fit(
	x=X_train_females,
	y=y_train_females,
    epochs=100,
    batch_size=64,
    validation_data= (X_test_females, y_test_females)
)

score = model.evaluate(
	x=X_test_females,
	y=y_test_females)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
9/9 [==============================] - 0s 44ms/step - loss: 2.1012 - accuracy: 0.1163 - val_loss: 2.0683 - val_accuracy: 0.1250
Epoch 2/100
9/9 [==============================] - 0s 25ms/step - loss: 2.0530 - accuracy: 0.1840 - val_loss: 1.9868 - val_accuracy: 0.3056
Epoch 3/100
9/9 [==============================] - 0s 25ms/step - loss: 1.8912 - accuracy: 0.3003 - val_loss: 1.7392 - val_accuracy: 0.3264
Epoch 4/100
9/9 [==============================] - 0s 25ms/step - loss: 1.6173 - accuracy: 0.4045 - val_loss: 1.4863 - val_accuracy: 0.4514
Epoch 5/100
9/9 [==============================] - 0s 25ms/step - loss: 1.3214 - accuracy: 0.5035 - val_loss: 1.2276 - val_accuracy: 0.5486
Epoch 6/100
9/9 [==============================] - 0s 25ms/step - loss: 1.0574 - accuracy: 0.6319 - val_loss: 1.1940 - val_accuracy: 0.5486
Epoch 7/100
9/9 [==============================] - 0s 25ms/step - loss: 0.9287 - accuracy: 0.6545 - val_loss: 1.1876 - val_accuracy: 0.5833
Epoch 8/100
9/9 [===